In [2]:
import pysam
import pandas as pd
import numpy as np
import glob
import time
import os

bam_paths = pd.read_csv(
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/split_bams.txt',
    header=None,
    names=['path'],
)

sample_to_mouse = {
    'L8TX_181211_01_A01':'Mouse1',
    'L8TX_181211_01_B01':'Mouse2',
    'L8TX_181211_01_C01':'Mouse2',
    'L8TX_181211_01_E01':'Mouse2',
    'L8TX_181211_01_F01':'Mouse1',
    'L8TX_181211_01_G12':'Mouse1',
    'L8TX_181211_01_H12':'Mouse1',
    'L8TX_190430_01_A08':'Mouse3',
    'L8TX_190430_01_B08':'Mouse3',
    'L8TX_190430_01_F08':'Mouse4',
    'L8TX_190430_01_G08':'Mouse4',
}

bam_paths['sample'] = bam_paths['path'].map(lambda p: os.path.basename(p).split('_chr')[0])
bam_paths['mouse'] = bam_paths['sample'].map(sample_to_mouse)
bam_paths['chr'] = 'chr'+bam_paths['path'].map(lambda p: os.path.basename(p).split('_chr')[1].split('_')[0])
bam_paths['ontology'] = bam_paths['path'].map(lambda p: '_'.join(os.path.basename(p).split('_chr')[1].split('_')[1:]))


#Sam flag meanings from https://broadinstitute.github.io/picard/explain-flags.html
bit_meanings = {
    'read paired':                               0b000000000001,
    'read mapped in proper pair':                0b000000000010,
    'read unmapped':                             0b000000000100,
    'mate unmapped':                             0b000000001000,
    'read reverse strand':                       0b000000010000,
    'mate reverse strand':                       0b000000100000,
    'first in pair':                             0b000001000000,
    'second in pair':                            0b000010000000,
    'not primary alignment':                     0b000100000000,
    'read fails platform/vendor quality checks': 0b001000000000,
    'read is PCR or optical duplicate':          0b010000000000,
    'supplementary alignment':                   0b100000000000,
}

#If any of these bits are set, reject the read
#can test if a flag is rejectable if flag&reject_flags != 0
reject_flags = (
    bit_meanings['read unmapped'] |
    bit_meanings['mate unmapped'] |
    bit_meanings['second in pair'] |
    bit_meanings['not primary alignment'] |
    bit_meanings['read fails platform/vendor quality checks'] |
    bit_meanings['supplementary alignment'] |
    bit_meanings['read is PCR or optical duplicate']
)


def pass_filter(read):
    cigar = read.cigar
    mq = read.mapping_quality
    read_length = read.query_length
    rejectable_flag = read.flag & reject_flags
    if cigar == [(0, read_length)] and mq == 255 and not rejectable_flag:
        return True

out_stem = '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/merged_by_mouse_celltype/{}_{}'

for (m,ont),g in bam_paths.groupby(['mouse','ontology']):
    #create a single merged file
    merge_file = out_stem.format(m,ont)
    pysam.merge('-f', merge_file, *g['path'].values)
    time.sleep(5)
    pysam.index(merge_file)
    
    #filter out the bad reads just as ReadZs does (copy and pasted)
    in_bam = pysam.AlignmentFile(merge_file)
    filt_out = merge_file.replace('.bam','_filtered.bam')
    out_bam = pysam.AlignmentFile(filt_out, 'wb', template=in_bam)
    
    #Iterate through all reads to determine if they pass filters, writing to new file
    for read in in_bam.fetch():
        if pass_filter(read):
            out_bam.write(read)
    
    #Then finally index the filtered bam
    in_bam.close()
    out_bam.close()
    pysam.index(filt_out)


,path,sample,mouse,chr,ontology
0,/scratch/groups/horence/rob/data/MERFISH_scRNA...,L8TX_181211_01_A01,Mouse1,chr10,L23_IT.bam
1,/scratch/groups/horence/rob/data/MERFISH_scRNA...,L8TX_181211_01_A01,Mouse1,chr10,L56_NP.bam
2,/scratch/groups/horence/rob/data/MERFISH_scRNA...,L8TX_181211_01_A01,Mouse1,chr10,L5_ET.bam
3,/scratch/groups/horence/rob/data/MERFISH_scRNA...,L8TX_181211_01_A01,Mouse1,chr10,L5_IT.bam
4,/scratch/groups/horence/rob/data/MERFISH_scRNA...,L8TX_181211_01_A01,Mouse1,chr10,L6_CT.bam
...,...,...,...,...,...
4835,/scratch/groups/horence/rob/data/MERFISH_scRNA...,L8TX_190430_01_G08,Mouse4,chrY,Sncg.bam
4836,/scratch/groups/horence/rob/data/MERFISH_scRNA...,L8TX_190430_01_G08,Mouse4,chrY,Sst.bam
4837,/scratch/groups/horence/rob/data/MERFISH_scRNA...,L8TX_190430_01_G08,Mouse4,chrY,VLMC.bam
4838,/scratch/groups/horence/rob/data/MERFISH_scRNA...,L8TX_190430_01_G08,Mouse4,chrY,Vip.bam


In [18]:
bam_paths[['mouse','ontology']].drop_duplicates().to_csv('/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/merged_by_mouse_celltype/mouse_onts.csv',index=False)

In [16]:
bam_paths['mouse'].nunique()

4

In [15]:
bam_paths['ontology'].nunique()

22

In [25]:
df = pd.read_csv(
    '/oak/stanford/groups/horence/rob/isoform_localizations/SRRS/plotting/buildup_plots/MERFISH_genes.bed',
    sep = ' ',
)
df['start'] -= 5000
df['end'] += 5000
df.to_csv(
    '/oak/stanford/groups/horence/rob/isoform_localizations/SRRS/plotting/buildup_plots/MERFISH_genes_permissive.bed',
    index=None,
    sep=' ',
)
df.head()

,#chr,start,end,gene,strand
0,chr1,5583466,5611131,Oprk1,+
1,chr1,12687277,12866192,Sulf1,+
2,chr1,12861549,12997650,Slco5a1,-
3,chr1,42690768,42708176,Pou3f3,+
4,chr1,56788981,56983650,Satb2,-


In [28]:
import itertools

In [34]:
bam_paths = [
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_A01.bam',
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_B01.bam',
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_C01.bam',
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_E01.bam',
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_F01.bam',
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_G12.bam',
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_H12.bam',
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_190430_01_A08.bam',
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_190430_01_B08.bam',
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_190430_01_F08.bam',
    '/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_190430_01_G08.bam',
]
chrs = df['#chr'].unique()



In [35]:
len(list(itertools.product(bam_paths,chrs)))

220

In [36]:
for p,c in itertools.product(bam_paths,chrs):
    print(p,c)


/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_A01.bam chr1
/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_A01.bam chr2
/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_A01.bam chr3
/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_A01.bam chr4
/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_A01.bam chr5
/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_A01.bam chr6
/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_A01.bam chr7
/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_181211_01_A01.bam chr8
/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8

In [37]:
bam = pysam.AlignmentFile('/scratch/groups/horence/rob/data/MERFISH_scRNAseq/10X_mapping/permissive_merfish_filt_L8TX_190430_01_G08.bam')
bam

In [38]:
bam.references

('chr1',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chrM',
 'chrX',
 'chrY',
 'JH584299.1',
 'GL456233.1',
 'JH584301.1',
 'GL456211.1',
 'GL456350.1',
 'JH584293.1',
 'GL456221.1',
 'JH584297.1',
 'JH584296.1',
 'GL456354.1',
 'JH584294.1',
 'JH584298.1',
 'JH584300.1',
 'GL456219.1',
 'GL456210.1',
 'JH584303.1',
 'JH584302.1',
 'GL456212.1',
 'JH584304.1',
 'GL456379.1',
 'GL456216.1',
 'GL456393.1',
 'GL456366.1',
 'GL456367.1',
 'GL456239.1',
 'GL456213.1',
 'GL456383.1',
 'GL456385.1',
 'GL456360.1',
 'GL456378.1',
 'GL456389.1',
 'GL456372.1',
 'GL456370.1',
 'GL456381.1',
 'GL456387.1',
 'GL456390.1',
 'GL456394.1',
 'GL456392.1',
 'GL456382.1',
 'GL456359.1',
 'GL456396.1',
 'GL456368.1',
 'JH584292.1',
 'JH584295.1')